<a href="https://colab.research.google.com/github/juliaviolet/BCCOptionsStrategies/blob/main/BCC_Options_Strategies_Modules_PEP8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction:**

This project presents an in-depth exploration of various hedging strategies, primarily focusing on the Bakshi, Cao, and Chen (BCC) model for valuing European options under conditions of stochastic volatility and jumps. Starting with model definitions, the notebook introduces the characteristic functions and option pricing techniques associated with the BCC model. Utilizing a dataset of option and underlying asset data, the model's application computes various Greeks, such as delta, vega, gamma, and theta, based on changes in the underlying asset's price. Subsequent sections delve into different hedging strategies, evaluating their effectiveness by calculating daily profits and running balances over a specific timeframe. These strategies range from delta hedging variations to approaches like the ratio spread, married put, and short strangle. Visualizations compare the final balances of each strategy, offering insights into their respective performances.


1. **Model Definitions**:
    - The code starts by defining the Bakshi, Cao, and Chen (BCC) model to value European options under stochastic volatility and jumps.
    - Several functions (`CIR_char_func_nojit`, `H93_char_func_nojit`, `M76_char_func_nojit`, etc.) are defined to compute the characteristic functions and option prices using the BCC model.

2. **Data Import**:
    - The dataset `sorted_updated_merged_data_fixed.csv` containing option and underlying asset data is imported.

3. **Model Application to Data**:
    - The BCC model is applied to the dataset to compute several Greeks (e.g., delta, vega, gamma, theta) and the model price of the option. The calculations use parameters that were derived in a separate notebook: BCC_Options_Strategies_8. (https://colab.research.google.com/drive/1jxHlUhLEG_aazBbT-k8WgoEMu7MdDgt1?usp=sharing)

    - These Greeks are calculated based on the changes in the underlying asset's price.
    - The computed values are added to the data, and the results are saved to `final_results_fixed.csv`.

4. **Hedging Metrics Calculation**:
    - The code calculates hedging metrics for various strategies such as delta hedging, delta-theta hedging, and shadow delta hedging.

    - The function `calculate_hedging_metrics` computes the daily profit and running balance for each strategy based on the daily hedge values.

    - For each strategy, the running balance is stored, representing the effectiveness of that strategy over the dataset's timeframe.

5. **Additional Hedging Strategies**:
    - The code also evaluates three other hedging strategies:
        - **Ratio Spread**: This strategy identifies the best ratio spread combination that maximizes the profit for both call and put options.
        - **Married Put**: This strategy evaluates the profitability of buying a put option as insurance for a stock position.
        - **Short Strangle**: This strategy calculates the profit or loss from selling out-of-the-money call and put options.

6. **Visualization and Results**:
    - The end balances of all strategies are aggregated and visualized in a bar chart using the Plotly library.
    - The chart provides a clear comparison of the effectiveness of each strategy.
    - The final balances for each strategy are printed out.
    - The running balances of the delta hedging strategies are visualized in a line chart.
   
The code provides an analysis of various hedging strategies applied to a dataset of options and an underlying asset. The BCC model is used to compute option prices and their Greeks, and the results are visualized to compare the effectiveness of each strategy.

In [1]:
model_content = """import numpy as np
import pandas as pd
from scipy.integrate import quad

# constants and global variables
time_bump = 1 / 365.25
r0 = -0.00067  # current risk-free rate


def CIR_char_func_nojit(u, T, kappa, theta, sigma_r, r0):
    gamma = np.sqrt(kappa**2 + 2*sigma_r**2*u*1j)
    A = (2 * gamma * np.exp((kappa + gamma) * T / 2) /
         (2 * gamma + (gamma + kappa) * (np.exp(gamma * T) - 1)))
    B = (2 * (np.exp(gamma * T) - 1) /
         (2 * gamma + (gamma + kappa) * (np.exp(gamma * T) - 1)))
    return np.exp(A * theta - B * r0)


def H93_char_func_nojit(u, T, r0, kappa_v, theta_v, sigma_v, rho, v0):
    c1 = kappa_v * theta_v
    c2 = -np.sqrt((rho * sigma_v * u * 1j - kappa_v)**2 -
                  sigma_v**2 * (-u * 1j - u**2))
    c3 = ((kappa_v - rho * sigma_v * u * 1j + c2) /
          (kappa_v - rho * sigma_v * u * 1j - c2))
    H1 = (r0 * u * 1j * T +
          (c1 / sigma_v**2) * ((kappa_v - rho * sigma_v * u * 1j + c2) * T -
                               2 * np.log((1 - c3 * np.exp(c2 * T)) / (1 - c3))))
    H2 = ((kappa_v - rho * sigma_v * u * 1j + c2) /
          sigma_v**2 * ((1 - np.exp(c2 * T)) / (1 - c3 * np.exp(c2 * T))))
    return np.exp(H1 + H2 * v0)


def M76_char_func_nojit(u, T, lamb, mu, delta):
    omega = -lamb * (np.exp(mu + 0.5 * delta**2) - 1)
    return np.exp((1j * u * omega + lamb *
                   (np.exp(1j * u * mu - u**2 * delta**2 * 0.5) - 1)) * T)


def BCC_char_func_nojit(u, T, r0, kappa_v, theta_v, sigma_v, rho, v0, lamb,
                        mu, delta, kappa, theta, sigma_r, muV, deltaV, rhoJ):
    CIR1 = CIR_char_func_nojit(u, T, kappa, theta, sigma_r, r0)
    BCC1 = H93_char_func_nojit(u, T, r0, kappa_v, theta_v, sigma_v, rho, v0)
    BCC2 = M76_char_func_nojit(u, T, lamb, mu, delta)
    jumpVolatilityPart = (lamb * (np.exp(1j * u * muV * deltaV) - 1) * T)
    return CIR1 * BCC1 * BCC2 * np.exp(jumpVolatilityPart * rhoJ)


def BCC_int_func_nojit(u, S0, K, T, r0, kappa_v, theta_v, sigma_v, rho, v0,
                       lamb, mu, delta, kappa, theta, sigma_r, muV, deltaV, rhoJ):
    char_func_value = BCC_char_func_nojit(u - 1j * 0.5, T, r0, kappa_v,
                                          theta_v, sigma_v, rho, v0, lamb, mu,
                                          delta, kappa, theta, sigma_r, muV, deltaV, rhoJ)
    return (1 / (u**2 + 0.25) *
            (np.exp(1j * u * np.log(S0 / K)) * char_func_value).real)


def BCC_call_value_nojit(S0, K, T, r0, kappa_v, theta_v, sigma_v, rho, v0,
                         lamb, mu, delta, kappa, theta, sigma_r, muV, deltaV, rhoJ):
    int_value = quad(lambda u: BCC_int_func_nojit(u, S0, K, T, r0, kappa_v,
                                                  theta_v, sigma_v, rho, v0,
                                                  lamb, mu, delta, kappa, theta,
                                                  sigma_r, muV, deltaV, rhoJ),
                     0, np.inf, limit=250)[0]
    return max(0, S0 - np.exp(-r0 * T) * np.sqrt(S0 * K) / np.pi * int_value)


def BCC_delta_nojit(S0, K, T, r0, kappa_v, theta_v, sigma_v, rho, v0, lamb,
                    mu, delta, kappa, theta, sigma_r, muV, deltaV, rhoJ,
                    price_change_abs, option_type):
    epsilon = price_change_abs if price_change_abs else 0.01
    price_plus = BCC_call_value_nojit(S0 + epsilon, K, T, r0, kappa_v, theta_v,
                                      sigma_v, rho, v0, lamb, mu, delta, kappa,
                                      theta, sigma_r, muV, deltaV, rhoJ)
    price_minus = BCC_call_value_nojit(S0 - epsilon, K, T, r0, kappa_v, theta_v,
                                       sigma_v, rho, v0, lamb, mu, delta, kappa,
                                       theta, sigma_r, muV, deltaV, rhoJ)
    if option_type == 'CALL':
        return (price_plus - price_minus) / (2 * epsilon)
    elif option_type == 'PUT ':
        return (price_minus - price_plus) / (2 * epsilon)
    else:
        return 0


def BCC_vega_nojit(S0, K, T, r0, kappa_v, theta_v, sigma_v, rho, v0, lamb,
                   mu, delta, kappa, theta, sigma_r, muV, deltaV, rhoJ,
                   price_change_abs):
    epsilon = 0.01
    vega_plus = BCC_call_value_nojit(S0, K, T, r0, kappa_v, theta_v,
                                     sigma_v + epsilon, rho, v0, lamb, mu,
                                     delta, kappa, theta, sigma_r, muV, deltaV, rhoJ)
    vega_minus = BCC_call_value_nojit(S0, K, T, r0, kappa_v, theta_v,
                                      sigma_v - epsilon, rho, v0, lamb, mu,
                                      delta, kappa, theta, sigma_r, muV, deltaV, rhoJ)
    return (vega_plus - vega_minus) / (2 * epsilon)


def BCC_theta_nojit(S0, K, T, r0, kappa_v, theta_v, sigma_v, rho, v0, lamb,
                    mu, delta, kappa, theta, sigma_r, muV, deltaV, rhoJ):
    theta_value = BCC_call_value_nojit(S0, K, T + time_bump, r0, kappa_v,
                                       theta_v, sigma_v, rho, v0, lamb, mu,
                                       delta, kappa, theta, sigma_r, muV, deltaV, rhoJ)
    theta_value_bumped = BCC_call_value_nojit(S0, K, T - time_bump, r0, kappa_v,
                                              theta_v, sigma_v, rho, v0, lamb, mu,
                                              delta, kappa, theta, sigma_r, muV, deltaV, rhoJ)
    return (theta_value_bumped - theta_value) / (2 * time_bump)


def BCC_gamma_nojit(S0, K, T, r0, kappa_v, theta_v, sigma_v, rho, v0, lamb,
                    mu, delta, kappa, theta, sigma_r, muV, deltaV, rhoJ,
                    price_change_abs, putcallind):
    epsilon = price_change_abs if price_change_abs else 0.01
    delta_current = BCC_delta_nojit(S0, K, T, r0, kappa_v, theta_v, sigma_v,
                                    rho, v0, lamb, mu, delta, kappa, theta,
                                    sigma_r, muV, deltaV, rhoJ, price_change_abs,
                                    putcallind)
    delta_plus = BCC_delta_nojit(S0 + epsilon, K, T, r0, kappa_v, theta_v,
                                 sigma_v, rho, v0, lamb, mu, delta, kappa, theta,
                                 sigma_r, muV, deltaV, rhoJ, price_change_abs,
                                 putcallind)
    delta_minus = BCC_delta_nojit(S0 - epsilon, K, T, r0, kappa_v, theta_v,
                                  sigma_v, rho, v0, lamb, mu, delta, kappa, theta,
                                  sigma_r, muV, deltaV, rhoJ, price_change_abs,
                                  putcallind)
    if putcallind == 'CALL':
        gamma = (delta_plus - 2 * delta_current + delta_minus) / (epsilon**2)
    elif putcallind == 'PUT ':
        gamma = (delta_minus - 2 * delta_current + delta_plus) / (epsilon**2)
    else:
        gamma = 0
    return abs(gamma)
    """


with open('model.py', 'w') as file:
    file.write(model_content)

In [2]:
utilities_content = """
import pandas as pd


def read_csv(file_name):
    return pd.read_csv(file_name)


def write_csv(df, file_name):
    df.to_csv(file_name, index=False)


def calculate_hedge_amounts(df):
    df['DELTA_HEDGE'] = df['DELTA'] * df['PRICE_CHANGE']
    df['VEGA_HEDGE'] = df['VEGA'] * df['PRICE_CHANGE']
    df['THETA_HEDGE'] = df['THETA'] * df['PRICE_CHANGE']
    df['GAMMA_HEDGE'] = df['GAMMA'] * df['PRICE_CHANGE']
"""


with open('utilities.py', 'w') as file:
    file.write(utilities_content)

In [13]:
main_content = """
from model import *
from utilities import *


def compute_and_save():

  # Read the data
  merged_data = read_csv('sorted_updated_merged_data_fixed.csv')

  # Compute model prices and Greek values using the BCC model
  # Use the model functions on the merged_data dataframe as shown in the provided code

  # Calculate the price change and its absolute value for the underlying asset
  merged_data['PRICE_CHANGE'] = merged_data['CF_CLOSE_y'].diff()
  merged_data['PRICE_CHANGE_ABS'] = merged_data['PRICE_CHANGE'].abs()

  # Compute model prices and Greek values using the BCC model
  merged_data['MODEL_PRICE'] = merged_data.apply(lambda row: BCC_call_value_nojit(row['CF_CLOSE_y'], row['STRIKE_PRC'], row['T'], -0.00067, 3.76807610e+00, 1.00000000e-03, 8.77986615e-03, 5.03063522e-01, 0.00000000e+00, 3.95979221e-01, 7.10854746e-02, 4.01539560e-02, 3.76807610e+00, 1.00000000e-03, 8.77986615e-03, 6.99623759e-02, 9.99763692e-02, 5.00002718e-01), axis=1)
  merged_data['DELTA'] = merged_data.apply(lambda row: BCC_delta_nojit(row['CF_CLOSE_y'], row['STRIKE_PRC'], row['T'], r0, 3.76807610e+00, 1.00000000e-03, 8.77986615e-03, 5.03063522e-01, 0.00000000e+00, 3.95979221e-01, 7.10854746e-02, 4.01539560e-02, 3.76807610e+00, 1.00000000e-03, 8.77986615e-03, 6.99623759e-02, 9.99763692e-02, 5.00002718e-01, row['PRICE_CHANGE_ABS'], row['PUTCALLIND']), axis=1)
  merged_data['VEGA'] = merged_data.apply(lambda row: BCC_vega_nojit(row['CF_CLOSE_y'], row['STRIKE_PRC'], row['T'], -0.00067, 3.76807610e+00, 1.00000000e-03, 8.77986615e-03, 5.03063522e-01, 0.00000000e+00, 3.95979221e-01, 7.10854746e-02, 4.01539560e-02, 3.76807610e+00, 1.00000000e-03, 8.77986615e-03, 6.99623759e-02, 9.99763692e-02, 5.00002718e-01, row['PRICE_CHANGE_ABS']), axis=1)
  merged_data['THETA'] = merged_data.apply(lambda row: BCC_theta_nojit(row['CF_CLOSE_y'], row['STRIKE_PRC'], row['T'], -0.00067, 3.76807610e+00, 1.00000000e-03, 8.77986615e-03, 5.03063522e-01, 0.00000000e+00, 3.95979221e-01, 7.10854746e-02, 4.01539560e-02, 3.76807610e+00, 1.00000000e-03, 8.77986615e-03, 6.99623759e-02, 9.99763692e-02, 5.00002718e-01), axis=1)
  merged_data['GAMMA'] = merged_data.apply(lambda row: BCC_gamma_nojit(row['CF_CLOSE_y'], row['STRIKE_PRC'], row['T'], -0.00067, 3.76807610e+00, 1.00000000e-03, 8.77986615e-03, 5.03063522e-01, 0.00000000e+00, 3.95979221e-01, 7.10854746e-02, 4.01539560e-02, 3.76807610e+00, 1.00000000e-03, 8.77986615e-03, 6.99623759e-02, 9.99763692e-02, 5.00002718e-01, row['PRICE_CHANGE_ABS'], row['PUTCALLIND']), axis=1)

  # Calculate hedge amounts
  calculate_hedge_amounts(merged_data)

  # Extract relevant columns for final results
  final_columns = ['Unnamed: 0', 'Instrument', 'CF_DATE', 'EXPIR_DATE', 'PUTCALLIND', 'STRIKE_PRC', 'CF_CLOSE_x', 'IMP_VOLT', 'CF_CLOSE_y',
                  'DELTA', 'VEGA', 'THETA', 'GAMMA', 'DELTA_HEDGE', 'VEGA_HEDGE', 'THETA_HEDGE', 'GAMMA_HEDGE']
  final_df = merged_data[final_columns]

  # Save the final results
  write_csv(final_df, 'final_results_fixed.csv')
  return merged_data
  """


with open('main.py', 'w') as file:
    file.write(main_content)


In [16]:
from main import *
import pandas as pd

# computing model prices and Greek values using the BCC model for the merged_data dataframe
merged_data = pd.read_csv('sorted_updated_merged_data_fixed.csv')

# calculate the price change and its absolute value for the underlying asset
merged_data['PRICE_CHANGE'] = merged_data['CF_CLOSE_y'].diff()
merged_data['PRICE_CHANGE_ABS'] = merged_data['PRICE_CHANGE'].abs()

# compute model prices and Greek values using the BCC model
merged_data['MODEL_PRICE'] = merged_data.apply(
    lambda row: BCC_call_value_nojit(
        row['CF_CLOSE_y'], row['STRIKE_PRC'], row['T'], -0.00067,
        3.76807610e+00, 1.00000000e-03, 8.77986615e-03, 5.03063522e-01,
        0.00000000e+00, 3.95979221e-01, 7.10854746e-02, 4.01539560e-02,
        3.76807610e+00, 1.00000000e-03, 8.77986615e-03, 6.99623759e-02,
        9.99763692e-02, 5.00002718e-01), axis=1)
merged_data['DELTA'] = merged_data.apply(
    lambda row: BCC_delta_nojit(
        row['CF_CLOSE_y'], row['STRIKE_PRC'], row['T'], r0, 3.76807610e+00,
        1.00000000e-03, 8.77986615e-03, 5.03063522e-01, 0.00000000e+00,
        3.95979221e-01, 7.10854746e-02, 4.01539560e-02, 3.76807610e+00,
        1.00000000e-03, 8.77986615e-03, 6.99623759e-02, 9.99763692e-02,
        5.00002718e-01, row['PRICE_CHANGE_ABS'], row['PUTCALLIND']), axis=1)
merged_data['VEGA'] = merged_data.apply(
    lambda row: BCC_vega_nojit(
        row['CF_CLOSE_y'], row['STRIKE_PRC'], row['T'], -0.00067, 3.76807610e+00,
        1.00000000e-03, 8.77986615e-03, 5.03063522e-01, 0.00000000e+00,
        3.95979221e-01, 7.10854746e-02, 4.01539560e-02, 3.76807610e+00,
        1.00000000e-03, 8.77986615e-03, 6.99623759e-02, 9.99763692e-02,
        5.00002718e-01, row['PRICE_CHANGE_ABS']), axis=1)
merged_data['THETA'] = merged_data.apply(
    lambda row: BCC_theta_nojit(
        row['CF_CLOSE_y'], row['STRIKE_PRC'], row['T'], -0.00067, 3.76807610e+00,
        1.00000000e-03, 8.77986615e-03, 5.03063522e-01, 0.00000000e+00,
        3.95979221e-01, 7.10854746e-02, 4.01539560e-02, 3.76807610e+00,
        1.00000000e-03, 8.77986615e-03, 6.99623759e-02, 9.99763692e-02,
        5.00002718e-01), axis=1)
merged_data['GAMMA'] = merged_data.apply(
    lambda row: BCC_gamma_nojit(
        row['CF_CLOSE_y'], row['STRIKE_PRC'], row['T'], -0.00067, 3.76807610e+00,
        1.00000000e-03, 8.77986615e-03, 5.03063522e-01, 0.00000000e+00,
        3.95979221e-01, 7.10854746e-02, 4.01539560e-02, 3.76807610e+00,
        1.00000000e-03, 8.77986615e-03, 6.99623759e-02, 9.99763692e-02,
        5.00002718e-01, row['PRICE_CHANGE_ABS'], row['PUTCALLIND']), axis=1)

/content/model.py:59: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.

/content/model.py:59: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.



In [17]:
import plotly.graph_objects as go

# Filter the data for call and put options
call_data = merged_data[merged_data['PUTCALLIND'] == 'CALL']
put_data = merged_data[merged_data['PUTCALLIND'] == 'PUT ']


# Function to create the figure for a given data subset for time
def create_figure(data, title):
    fig = go.Figure()

    # Hovertemplate for Model Price
    hovertemplate_model = ('<span style="color:royalblue"><b>Model Price</b>: %{y:.4f}</span><br>'
                           '<span style="color:red"><b>Delta</b>: %{customdata[1]:.4f}</span><br>'
                           '<span style="color:green"><b>Vega</b>: %{customdata[2]:.4f}</span><br>'
                           '<span style="color:orange"><b>Theta</b>: %{customdata[3]:.4f}</span><br>'
                           '<span style="color:purple"><b>Gamma</b>: %{customdata[4]:.4f}</span><br>'
                           '<extra></extra>')

    # Hovertemplate for Underlying Asset
    hovertemplate_asset = ('<span style="color:gray"><b>Underlying Asset</b>: %{y:.4f}</span><br>'
                           '<extra></extra>')

    # Model Price trace
    fig.add_trace(go.Scatter(x=data['CF_DATE'], y=data['MODEL_PRICE'],
                             mode='lines',
                             name='Model Price',
                             line=dict(color='royalblue'),
                             customdata=data[['MODEL_PRICE', 'DELTA', 'VEGA', 'THETA', 'GAMMA']].values,
                             hovertemplate=hovertemplate_model))

    # Underlying Asset trace
    fig.add_trace(go.Scatter(x=data['CF_DATE'], y=data['CF_CLOSE_y'],
                             mode='lines',
                             name='Underlying Asset',
                             line=dict(color='gray'),
                             hovertemplate=hovertemplate_asset))

    # Greeks (with hoverinfo set to 'skip')
    fig.add_trace(go.Scatter(x=data['CF_DATE'], y=data['DELTA'],
                             mode='lines',
                             name='Delta',
                             line=dict(color='red'),
                             hoverinfo='skip'))
    fig.add_trace(go.Scatter(x=data['CF_DATE'], y=data['VEGA'],
                             mode='lines',
                             name='Vega',
                             line=dict(color='green'),
                             hoverinfo='skip'))
    fig.add_trace(go.Scatter(x=data['CF_DATE'], y=data['THETA'],
                             mode='lines',
                             name='Theta',
                             line=dict(color='orange'),
                             hoverinfo='skip'))
    fig.add_trace(go.Scatter(x=data['CF_DATE'], y=data['GAMMA'],
                             mode='lines',
                             name='Gamma',
                             line=dict(color='purple'),
                             hoverinfo='skip'))

    # Update the layout
    fig.update_layout(title=title,
                      xaxis_title='Date',
                      yaxis_title='Value',
                      template='plotly',
                      hovermode='x unified')
    fig.show()


# Create figures for call and put options over time
create_figure(call_data, 'Call Options: Model, Underlying Asset, and Greeks Over Time')
create_figure(put_data, 'Put Options: Model, Underlying Asset, and Greeks Over Time')


**Delta (Δ)**:
Delta represents the sensitivity of an option's price to a change in the price of the underlying asset. In the provided code, delta is computed by evaluating the option price for positive and negative changes in the underlying asset's price. In the plot, the delta line gives an idea of how sensitive the option's price is to movements in the underlying asset's price over time.

**Vega (ν)**:
Vega measures the sensitivity of the option's price to changes in volatility. The code calculates vega by evaluating the option price for small increases and decreases in the volatility parameter. The vega line in the plot highlights how sensitive the option's price is to volatility changes over different times.

**Gamma (Γ)**:
Gamma provides insight into the rate of change of delta with respect to changes in the underlying asset's price. It is essentially the second derivative of the option price concerning the underlying price. The code computes gamma by evaluating delta for tiny positive and negative changes in the underlying asset's price. The gamma line in the plot showcases how this sensitivity evolves over time.

**Theta (Θ)**:
Theta gauges the sensitivity of the option's price to the passage of time, representing time decay. In the code, theta is computed by evaluating the option price for small advancements and retractions in time. The theta line in the plot offers a visual representation of how the option's price is impacted by time decay.

In [19]:
import pandas as pd
import plotly.graph_objects as go

# Load the data
data = pd.read_csv('final_results_fixed_check_vega.csv')

# Filter the data for call and put options
call_data = data[data['PUTCALLIND'] == 'CALL']
put_data = data[data['PUTCALLIND'] == 'PUT ']


# Create a plotting function for each Greek
def plot_greek(greek_name):
    fig = go.Figure()

    # Define hovertemplate to show the 'Unnamed: 0' value, the date, and the greek value
    hovertemplate = "<b>Date</b>: %{customdata[1]}<br><b>" + greek_name + "</b>: %{y:.4f}<br><b>Option Row Number</b>: %{customdata[0]}<extra></extra>"

    # Add traces for Call and Put options
    fig.add_trace(go.Scatter(x=call_data['CF_DATE'],
                             y=call_data[greek_name],
                             customdata=call_data[['Unnamed: 0', 'CF_DATE']],
                             mode='lines',
                             name=f'CALL {greek_name}',
                             hovertemplate=hovertemplate))

    fig.add_trace(go.Scatter(x=put_data['CF_DATE'],
                             y=put_data[greek_name],
                             customdata=put_data[['Unnamed: 0', 'CF_DATE']],
                             mode='lines',
                             name=f'PUT {greek_name}',
                             hovertemplate=hovertemplate))

    # Update the layout
    fig.update_layout(title=f'{greek_name} Over Time for CALL and PUT Options',
                      xaxis_title='Date',
                      yaxis_title=greek_name,
                      template='plotly')
    fig.show()


# Generate the plots
plot_greek('DELTA')
plot_greek('VEGA')
plot_greek('THETA')
plot_greek('GAMMA')


**Delta (Δ):** Delta remains relatively stable during some periods and significantly fluctuates during others. When Delta is close to 1 (for call options) or -1 (for put options), it indicates deep in-the-money options. Conversely, values close to 0 indicate out-of-the-money options.

**Vega (ν):**
High vega values for both calls and puts suggest the market is expecting significant volatility in the future, making the options more sensitive to changes in implied volatility.

**Theta (Θ):**
Theta values are mostly negative, which is expected since options tend to lose value as time progresses (time decay).
The magnitude of Theta seems to be increasing over time, implying that as options approach their expiration dates, time decay accelerates. This is a characteristic of theta.

**Gamma (Γ):** Gamma measures the rate of change in delta with respect to the underlying asset's price. The relative stability suggests that the sensitivity of delta to price changes is consistent. Lower gamma values for deep in-the-money or out-of-the-money options mean that delta is relatively stable for these options because it has approached its bounds.

**Hedging Strategies Overview**

Each strategy calculates daily profits, adjusting for transaction costs and dynamic market conditions:

1. **Delta Theta**: Incorporates delta, vega, theta, and gamma.
2. **Delta**: Focuses on delta, vega, and gamma, excluding theta.
3. **Shadow Delta**: Adjusts delta based on daily price and volatility changes, and includes theta.
4. **Shadow Delta Without Theta**: An iteration of the shadow delta strategy without the theta component.
5. **Adjusted Shadow Delta**: Modifies the delta with a volatility smile factor, reflecting the difference between the option's strike price and the actual stock price.
6. **Adjusted Shadow Delta Without Theta**: A variant of the adjusted shadow delta that omits theta.
7. **Taleb's Adjusted Shadow Delta**: Uses the discrete measurement of gamma hedge as introduced in the linked article, adjusting the delta hedge with a gamma adjustment which is a function of the square of the price change.
8. **Taleb's Adjusted Shadow Delta Without Theta**: Similar to Taleb's Adjusted Shadow Delta but excludes the theta component for the adjustment.
9. **Ratio Spread**: Seeks the optimal combination of options to maximize profits.
10. **Married Put**: Involves buying a put option for a stock that's already owned to provide protection against potential stock price declines.
11. **Short Strangle**: Sells out-of-the-money call and put options, aiming to profit when the underlying asset's price remains between the two strike prices.


A note on strategies 7 and 8:

The provided article (linked below) discusses the concept of shadow gamma. It refers to the discrete measurement of the gamma hedge in the context of hedging options. The article suggests a methodology to calculate the shadow gamma by adjusting the delta hedge using a gamma adjustment, which is a function of the square of the price change. In the provided Python code, we see the implementation of various hedging strategies, including the adjusted shadow delta approach, which closely aligns with the article's content. The function `compute_taleb_adjusted_shadow_delta` captures this approach, wherein the delta hedge is adjusted using the gamma hedge and the square of the price change. This adjustment is used to determine the impact of the option's second-order Greek (gamma) on the delta hedge.

https://financetrainingcourse.com/education/2014/06/calculating-shadow-gamma-talebs-approach-for-the-second-order-option-greek/#:~:text=In%20this%20discrete%20measurement%2C%20gamma,down%20by%20an%20incremental%20value.

In [20]:
# data_loader.py


data_loader_content = """
import pandas as pd


def load_final_results():
    return pd.read_csv('final_results_fixed_price_rounded_check_vega.csv')

def load_combined_hedging_metrics():
    return pd.read_csv('combined_hedging_metrics_rounded_check_vega.csv')

def load_sorted_updated_merged_data():
    return pd.read_csv('sorted_updated_merged_data_fixed.csv')
"""


with open('data_loader.py', 'w') as file:
    file.write(data_loader_content)

In [31]:
# strategies.py
strategies_content = """

import numpy as np
import pandas as pd


# Function to compute the adjusted shadow delta based on Taleb's approach
def compute_taleb_adjusted_shadow_delta(row, matching_row, include_theta=True):
    gamma_hedge = matching_row['GAMMA_HEDGE']
    delta_hedge = matching_row['DELTA_HEDGE']
    vega_hedge = matching_row['VEGA_HEDGE']
    theta_hedge = (matching_row['THETA_HEDGE'] if include_theta else 0.0)

    # Taleb's adjustment
    gamma_adjustment = (gamma_hedge * (row['Price_Change']**2) / 2)
    delta_hedge += gamma_adjustment

    return round(
        delta_hedge + gamma_hedge * row['Price_Change'] +
        vega_hedge * row['Volatility_Change'] + theta_hedge
    )


# Original function to compute the adjusted shadow delta
# taking into account the volatility smile
def compute_adjusted_shadow_delta(row, matching_row, include_theta=True):
    adj_factor = 0.05  # Adjust delta based on the volatility smile
    delta_hedge = matching_row['DELTA_HEDGE']
    gamma_hedge = matching_row['GAMMA_HEDGE']
    vega_hedge = matching_row['VEGA_HEDGE']
    theta_hedge = (matching_row['THETA_HEDGE'] if include_theta else 0.0)
    price_diff = row['CF_CLOSE_y'] - matching_row['STRIKE_PRC']

    # Adjust delta based on the volatility smile
    if matching_row['PUTCALLIND'] == 'CALL':
        if price_diff > 0:
            delta_hedge -= adj_factor * abs(price_diff)
        else:
            delta_hedge += adj_factor * abs(price_diff)
    else:
        if price_diff < 0:
            delta_hedge += adj_factor * abs(price_diff)
        else:
            delta_hedge -= adj_factor * abs(price_diff)

    return round(
        delta_hedge + gamma_hedge * row['Price_Change'] +
        vega_hedge * row['Volatility_Change'] + theta_hedge
    )


# Function to calculate hedging metrics based on different strategies
def calculate_hedging_metrics(
        final_results_data,
        strategies=['DELTA', 'VEGA', 'THETA', 'GAMMA'],
        shadow=False, adjust_for_smile=False,
        use_taleb_adjustment=False):

    final_results_data = final_results_data.iloc[1:].reset_index(drop=True)
    final_results_data['Volatility_Change'] = (
        final_results_data['IMP_VOLT'].diff().fillna(0))

    initial_portfolio_value = 10000.00
    cost_per_contract = 0.75
    running_balance = initial_portfolio_value
    hedging_metrics_df = pd.DataFrame()

    for index, row in final_results_data.iterrows():
        metrics_row = row.to_dict()
        daily_profit = 0.0

        if shadow:
            matching_rows = (
                final_results_data[final_results_data['Unnamed: 0'] ==
                row['Unnamed: 0']])
            if not matching_rows.empty:
                matching_row = matching_rows.iloc[0]
                if use_taleb_adjustment:
                    delta_value = compute_taleb_adjusted_shadow_delta(
                        row, matching_row, 'THETA' in strategies)
                elif adjust_for_smile:
                    delta_value = compute_adjusted_shadow_delta(
                        row, matching_row, 'THETA' in strategies)
                else:
                    delta_value = (row['DELTA_HEDGE'] +
                                   row['GAMMA_HEDGE'] * row['Price_Change'] +
                                   row['VEGA_HEDGE'] * row['Volatility_Change'])
                    if 'THETA' in strategies:
                        delta_value += row['THETA_HEDGE']
                hedge_value = round(delta_value)
            else:
                hedge_value = sum(
                    row[strategy + '_HEDGE_ROUND'] for strategy in strategies)
        else:
            hedge_value = sum(
                row[strategy + '_HEDGE_ROUND'] for strategy in strategies)

        transaction_value = -1 * hedge_value * row['CF_CLOSE_x']
        daily_buy_cost = (
            abs(transaction_value) * cost_per_contract
            if transaction_value > 0 else 0)
        daily_sell_cost = (
            abs(transaction_value) * cost_per_contract
            if transaction_value < 0 else 0)
        total_cost_revenue = (
            transaction_value - daily_buy_cost - daily_sell_cost)

        if total_cost_revenue > 0:
            daily_profit += total_cost_revenue

        running_balance += daily_profit
        metrics_row["Running_Balance"] = running_balance
        hedging_metrics_df = pd.concat(
            [hedging_metrics_df, pd.DataFrame([metrics_row])], ignore_index=True)

    return hedging_metrics_df


# Ratio Spread Strategy: Aim to find the best combination of options
# to maximize profit
def find_ratio_spread_combination(options_data):
    best_combination = None
    best_profit = float('-inf')
    best_ratio = None
    options_list = options_data.to_dict(orient='records')
    for i, option_1 in enumerate(options_list):
        for j, option_2 in enumerate(options_list[i+1:], start=i+1):
            for ratio in range(1, 101):
                profit = ratio * option_1['CF_CLOSE_x'] - option_2['CF_CLOSE_x']
                if profit > best_profit:
                    best_profit = profit
                    best_combination = [option_1, option_2]
                    best_ratio = ratio
    return best_combination, best_ratio, best_profit


# Short Strangle Strategy: Involves selling out-of-the-money call and put options
def select_out_of_the_money_call(group):
    return group[group['STRIKE_PRC'] < group['CF_CLOSE_y']].nlargest(1, 'STRIKE_PRC')

def select_out_of_the_money_put(group):
    return group[group['STRIKE_PRC'] > group['CF_CLOSE_y']].nsmallest(1, 'STRIKE_PRC')
"""


with open('strategies.py', 'w') as file:
    file.write(strategies_content)


In [22]:
# visualization.py
visualization_content = """
import plotly.graph_objects as go


def plot_strategy_balances(sorted_balances):
    strategies = list(sorted_balances.keys())
    values = list(sorted_balances.values())
    fig = go.Figure(data=[go.Bar(
        x=strategies,
        y=values,
        marker=dict(
            color=values,
            colorscale='viridis',
            colorbar=dict(title='Balance')
        )
    )])
    fig.update_layout(
        title="Balances Across Different Hedging Strategies",
        xaxis_title="Strategy",
        yaxis_title="Balance",
        template="plotly",
        xaxis_tickangle=-45,
        yaxis=dict(tickformat="$,.2f")
    )
    fig.show()

"""


with open('visualization.py', 'w') as file:
    file.write(visualization_content)


In [34]:
# main_2.py


main_2_content = """
from data_loader import *
from strategies import *
from visualization import plot_strategy_balances


def compute_strategies(final_results_data, combined_hedging_metrics,
                       sorted_updated_merged_data):
    # Filter and sort the data, then calculate profits based on the strategy
    df_clean = combined_hedging_metrics.dropna(subset=['CF_CLOSE_x', 'CF_CLOSE_y'])
    calls_df_clean = df_clean[df_clean['PUTCALLIND'] == 'CALL']
    puts_df_clean = df_clean[df_clean['PUTCALLIND'] == 'PUT ']
    ootm_calls = calls_df_clean.groupby('CF_DATE').apply(select_out_of_the_money_call)
    ootm_calls = ootm_calls.dropna().reset_index(drop=True)
    ootm_puts = puts_df_clean.groupby('CF_DATE').apply(select_out_of_the_money_put)
    ootm_puts = ootm_puts.dropna().reset_index(drop=True)
    long_strangle_df = pd.merge(ootm_calls, ootm_puts, on='CF_DATE',
                                suffixes=('_call', '_put'))
    df_sorted = df_clean.sort_values(by="CF_DATE")
    df_sorted['CF_CLOSE_y_next'] = df_sorted['CF_CLOSE_y'].shift(-1)
    long_strangle_next = pd.merge(long_strangle_df,
                                  df_sorted[['CF_DATE', 'CF_CLOSE_y_next']],
                                  on='CF_DATE', how='left')

    condition1 = long_strangle_next['CF_CLOSE_y_next'] > long_strangle_next['STRIKE_PRC_call']
    value1 = (-((long_strangle_next['CF_CLOSE_y_next'] -
                long_strangle_next['STRIKE_PRC_call']) * 100) +
              (long_strangle_next['CF_CLOSE_x_call'] * 100))
    long_strangle_next['profit_from_call_sold'] = np.where(condition1, value1,
                                                           long_strangle_next['CF_CLOSE_x_call'] * 100)

    condition2 = long_strangle_next['CF_CLOSE_y_next'] < long_strangle_next['STRIKE_PRC_put']
    value2 = (-((long_strangle_next['STRIKE_PRC_put'] -
                long_strangle_next['CF_CLOSE_y_next']) * 100) +
              (long_strangle_next['CF_CLOSE_x_put'] * 100))
    long_strangle_next['profit_from_put_sold'] = np.where(condition2, value2,
                                                         long_strangle_next['CF_CLOSE_x_put'] * 100)

    long_strangle_next['total_profit_loss_sold'] = (long_strangle_next['profit_from_call_sold'] +
                                                   long_strangle_next['profit_from_put_sold'])

    strategies = ['DELTA', 'VEGA', 'THETA', 'GAMMA']
    taleb_metrics = calculate_hedging_metrics(final_results_data, strategies,
                                              shadow=True, use_taleb_adjustment=True)
    taleb_no_theta_metrics = calculate_hedging_metrics(final_results_data,
                                                       strategies[:-1],
                                                       shadow=True,
                                                       use_taleb_adjustment=True)
    delta_theta_metrics = calculate_hedging_metrics(final_results_data, strategies)
    delta_metrics = calculate_hedging_metrics(final_results_data, strategies[:-1])
    shadow_delta_metrics = calculate_hedging_metrics(final_results_data, strategies,
                                                     shadow=True)
    shadow_no_theta_metrics = calculate_hedging_metrics(final_results_data, strategies[:-1],
                                                        shadow=True)
    adj_shadow_delta_metrics = calculate_hedging_metrics(final_results_data, strategies,
                                                         shadow=True, adjust_for_smile=True)
    adj_shadow_no_theta_metrics = calculate_hedging_metrics(final_results_data, strategies[:-1],
                                                            shadow=True, adjust_for_smile=True)
    filtered_data = sorted_updated_merged_data[(sorted_updated_merged_data['EXPIR_DATE'] == '2020-06-19') &
                                               (sorted_updated_merged_data['IMP_VOLT'] > 0)]
    call_data_ratio = filtered_data[filtered_data['PUTCALLIND'] == 'CALL'].sort_values(by='STRIKE_PRC')
    put_data_ratio = filtered_data[filtered_data['PUTCALLIND'] == 'PUT '].sort_values(by='STRIKE_PRC')
    best_call_combination, best_call_ratio, best_call_profit = find_ratio_spread_combination(call_data_ratio)
    best_put_combination, best_put_ratio, best_put_profit = find_ratio_spread_combination(put_data_ratio)

    put_options = combined_hedging_metrics[combined_hedging_metrics['PUTCALLIND'] == 'PUT '].copy()
    put_options['stock_cost'] = put_options['CF_CLOSE_y'] * 100
    put_options['option_cost'] = put_options['CF_CLOSE_x'] * 1
    put_options['total_cost'] = put_options['stock_cost'] + put_options['option_cost']
    put_options['stock_profit_next_day'] = (put_options['CF_CLOSE_y'].shift(-1) * 100) -
                                           put_options['stock_cost']
    put_options['stock_price_next_day'] = put_options['CF_CLOSE_y'].shift(-1)
    profit_func = lambda row: (max(0, row['STRIKE_PRC'] - row['stock_price_next_day']) * 100 -
                               row['option_cost'])
    put_options['option_profit'] = put_options.apply(profit_func, axis=1)
    put_options['net_profit'] = put_options['stock_profit_next_day'] +
                                put_options['option_profit']

    balances = {
        "Delta Theta": delta_theta_metrics['Running_Balance'].iloc[-1],
        "Delta": delta_metrics['Running_Balance'].iloc[-1],
        "Shadow Delta": shadow_delta_metrics['Running_Balance'].iloc[-1],
        "Shadow Delta Without Theta": shadow_no_theta_metrics['Running_Balance'].iloc[-1],
        "Adjusted Shadow Delta": adj_shadow_delta_metrics['Running_Balance'].iloc[-1],
        "Adjusted Shadow Delta Without Theta": adj_shadow_no_theta_metrics['Running_Balance'].iloc[-1],
        "Taleb Shadow Delta": taleb_metrics['Running_Balance'].iloc[-1],
        "Taleb Shadow Delta Without Theta": taleb_no_theta_metrics['Running_Balance'].iloc[-1],
        "Ratio Spread (Calls)": best_call_profit,
        "Ratio Spread (Puts)": best_put_profit,
        "Married Put": put_options['net_profit'].sum(),
        "Short Strangle": long_strangle_next['total_profit_loss_sold'].sum()
    }

    sorted_balances = {k: v for k, v in sorted(balances.items(),
                                               key=lambda item: item[1],
                                               reverse=True)}
    return sorted_balances


if __name__ == "__main__":
    final_results_data = load_final_results()
    combined_hedging_metrics = load_combined_hedging_metrics()
    sorted_updated_merged_data = load_sorted_updated_merged_data()

    sorted_balances = compute_strategies(final_results_data,
                                         combined_hedging_metrics,
                                         sorted_updated_merged_data)

    plot_strategy_balances(sorted_balances)
"""


with open('main_2.py', 'w') as file:
    file.write(main_2_content)

# Return paths for the files saved
saved_files = {
    "data_loader.py": "data_loader.py",
    "strategies.py": "strategies.py",
    "visualization.py": "visualization.py",
    "main_2.py": "main_2.py"
}

saved_files


{'data_loader.py': 'data_loader.py',
 'strategies.py': 'strategies.py',
 'visualization.py': 'visualization.py',
 'main_2.py': 'main_2.py'}

In [35]:
from main_2 import compute_strategies
from visualization import plot_strategy_balances
from data_loader import load_final_results, load_combined_hedging_metrics, load_sorted_updated_merged_data

final_results_data = load_final_results()
combined_hedging_metrics = load_combined_hedging_metrics()
sorted_updated_merged_data = load_sorted_updated_merged_data()

sorted_balances = compute_strategies(final_results_data, combined_hedging_metrics, sorted_updated_merged_data)

# Print the final balance for each strategy
for strategy, balance in sorted_balances.items():
    print(f"{strategy}: ${balance:,.2f}")

plot_strategy_balances(sorted_balances)


Adjusted Shadow Delta: $158,024,903.20
Shadow Delta: $147,421,901.57
Taleb Shadow Delta: $147,299,669.05
Adjusted Shadow Delta Without Theta: $85,788,036.90
Delta Theta: $80,014,712.25
Shadow Delta Without Theta: $75,069,624.08
Taleb Shadow Delta Without Theta: $74,957,085.98
Married Put: $23,812,309.20
Short Strangle: $18,139,150.00
Delta: $2,767,251.33
Ratio Spread (Calls): $832,329.90
Ratio Spread (Puts): $810,250.20


In [38]:
from main_2 import compute_strategies, calculate_hedging_metrics


def fetch_plotting_data(final_results_data, combined_hedging_metrics):
    # compute the metrics dataframes
    strategies1 = ['DELTA', 'VEGA', 'THETA', 'GAMMA']
    strategies2 = ['DELTA', 'VEGA', 'GAMMA']

    delta_theta_metrics = calculate_hedging_metrics(final_results_data,
                                                    strategies=strategies1)
    delta_metrics = calculate_hedging_metrics(final_results_data,
                                              strategies=strategies2)
    shadow_delta_metrics = calculate_hedging_metrics(
        final_results_data, strategies=strategies1, shadow=True
    )
    shadow_delta_without_theta_metrics = calculate_hedging_metrics(
        final_results_data, strategies=strategies2, shadow=True
    )
    adjusted_shadow_delta_metrics = calculate_hedging_metrics(
        final_results_data, strategies=strategies1, shadow=True,
        adjust_for_smile=True
    )
    adjusted_shadow_delta_without_theta_metrics = calculate_hedging_metrics(
        final_results_data, strategies=strategies2, shadow=True,
        adjust_for_smile=True
    )
    taleb_shadow_delta_metrics = calculate_hedging_metrics(
        final_results_data, strategies=strategies1, shadow=True,
        use_taleb_adjustment=True
    )
    taleb_shadow_delta_without_theta_metrics = calculate_hedging_metrics(
        final_results_data, strategies=strategies2, shadow=True,
        use_taleb_adjustment=True
    )

    return {
        "Delta Theta": delta_theta_metrics,
        "Delta": delta_metrics,
        "Shadow Delta": shadow_delta_metrics,
        "Shadow Delta Without Theta": shadow_delta_without_theta_metrics,
        "Adjusted Shadow Delta": adjusted_shadow_delta_metrics,
        "Adjusted Shadow Delta Without Theta":
        adjusted_shadow_delta_without_theta_metrics,
        "Taleb Shadow Delta": taleb_shadow_delta_metrics,
        "Taleb Shadow Delta Without Theta":
        taleb_shadow_delta_without_theta_metrics
    }


# fetch the dataframes for plotting
strategy_metrics = fetch_plotting_data(final_results_data,
                                       combined_hedging_metrics)


In [40]:
import plotly.graph_objects as go

# Extracting metrics dataframes from strategy_metrics
delta_theta_metrics = strategy_metrics["Delta Theta"]
shadow_delta_metrics = strategy_metrics["Shadow Delta"]
adjusted_shadow_delta_metrics = strategy_metrics["Adjusted Shadow Delta"]
taleb_shadow_delta_metrics = strategy_metrics["Taleb Shadow Delta"]
delta_metrics = strategy_metrics["Delta"]
shadow_delta_wo_theta_metrics = strategy_metrics["Shadow Delta Without Theta"]
adjusted_shadow_delta_wo_theta_metrics = strategy_metrics[
    "Adjusted Shadow Delta Without Theta"
]
taleb_shadow_delta_wo_theta_metrics = strategy_metrics[
    "Taleb Shadow Delta Without Theta"
]

# Extracting the 'Running_Balance' columns and the 'CF_DATE' column
dates = delta_theta_metrics['CF_DATE']  # Assuming 'CF_DATE' is the date column

# Store metrics dataframes and their names in lists for including/excluding theta
metrics_including_theta = [
    ("Delta Theta", delta_theta_metrics),
    ("Shadow Delta", shadow_delta_metrics),
    ("Adjusted Shadow Delta", adjusted_shadow_delta_metrics),
    ("Taleb Shadow Delta", taleb_shadow_delta_metrics)
]

metrics_excluding_theta = [
    ("Delta", delta_metrics),
    ("Shadow Delta Without Theta", shadow_delta_wo_theta_metrics),
    ("Adjusted Shadow Delta Without Theta", adjusted_shadow_delta_wo_theta_metrics),
    ("Taleb Shadow Delta Without Theta", taleb_shadow_delta_wo_theta_metrics)
]

# Sort these lists by the final balance of each dataframe in descending order
metrics_including_theta_sorted = sorted(
    metrics_including_theta,
    key=lambda x: x[1]['Running_Balance'].iloc[-1],
    reverse=True
)
metrics_excluding_theta_sorted = sorted(
    metrics_excluding_theta,
    key=lambda x: x[1]['Running_Balance'].iloc[-1],
    reverse=True
)


# Create plots using the sorted lists
def create_figure(metrics_sorted, title):
    plot_data = [
        go.Scatter(
            x=dates,
            y=metric_df['Running_Balance'],
            mode='lines',
            name=name,
            hovertemplate='<b>%{fullData.name} Balance:</b> '
                          '%{y:$,.2f}<extra></extra>'
        ) for name, metric_df in metrics_sorted
    ]
    fig = go.Figure(data=plot_data)
    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title="Running Balance",
        template="plotly",
        hovermode='x unified'
    )
    fig.show()


create_figure(metrics_including_theta_sorted,
              "Running Balance Over Time (Including Theta)")
create_figure(metrics_excluding_theta_sorted,
              "Running Balance Over Time (Excluding Theta)")


**Conclusion:**

The hedging strategies outlined in this project offer a toolkit for managing portfolio risks in the options market. By leveraging the BCC model's capabilities, it has computed the Greeks, which play an instrumental role in these strategies. The final line plot visualization displays a clear comparative view of each strategy's effectiveness over time. Each strategy has its own set of advantages and use-cases, helping investors make informed decisions based on varying market conditions and risk appetites.